In [ ]:
from agent.TradingAgent import TradingAgent
import pandas as pd
import numpy as np
import os
from contributed_traders.util import get_file

class SimpleAgent(TradingAgent):
    """
    Simple Trading Agent that compares the past mid-price observations and places a
    buy limit order if the first window mid-price exponential average >= the second window mid-price exponential average or a
    sell limit order if the first window mid-price exponential average < the second window mid-price exponential average
    """

    def __init__(self, id, name, type, symbol, starting_cash,
                 min_size, max_size, wake_up_freq='60s',
                 log_orders=False, random_state=None):

        super().__init__(id, name, type, starting_cash=starting_cash, log_orders=log_orders, random_state=random_state)
        self.symbol = symbol
        self.min_size = min_size  # Minimum order size
        self.max_size = max_size  # Maximum order size
        self.size = self.random_state.randint(self.min_size, self.max_size)
        self.wake_up_freq = wake_up_freq
        self.mid_list, self.avg_win1_list, self.avg_win2_list = [], [], []
        self.log_orders = log_orders
        self.state = "AWAITING_WAKEUP"
        #self.window1 = 100 
        #self.window2 = 5 

    def kernelStarting(self, startTime):
        super().kernelStarting(startTime)
        # Read in the configuration through util
        with open(get_file('simple_agent.cfg'), 'r') as f:
            self.window1, self.window2 = [int(w) for w in f.readline().split()]
        #print(f"{self.window1} {self.window2}")

    def wakeup(self, currentTime):
        """ Agent wakeup is determined by self.wake_up_freq """
        can_trade = super().wakeup(currentTime)
        if not can_trade: return
        self.getCurrentSpread(self.symbol)
        self.state = 'AWAITING_SPREAD'

    def dump_shares(self):
        # get rid of any outstanding shares we have
        if self.symbol in self.holdings and len(self.orders) == 0:
            order_size = self.holdings[self.symbol]
            bid, _, ask, _ = self.getKnownBidAsk(self.symbol)
            if bid:
                self.placeLimitOrder(self.symbol, quantity=order_size, is_buy_order=False, limit_price=0)

    def receiveMessage(self, currentTime, msg):
        """ Momentum agent actions are determined after obtaining the best bid and ask in the LOB """
        super().receiveMessage(currentTime, msg)
        if self.state == 'AWAITING_SPREAD' and msg.body['msg'] == 'QUERY_SPREAD':
            dt = (self.mkt_close - currentTime) / np.timedelta64(1, 'm')
            if dt < 25:
                self.dump_shares()
            else:
                bid, _, ask, _ = self.getKnownBidAsk(self.symbol)
                if bid and ask:
                    self.mid_list.append((bid + ask) / 2)
                    if len(self.mid_list) > self.window1: self.avg_win1_list.append(pd.Series(self.mid_list).ewm(span=self.window1).mean().values[-1].round(2))
                    if len(self.mid_list) > self.window2: self.avg_win2_list.append(pd.Series(self.mid_list).ewm(span=self.window2).mean().values[-1].round(2))
                    if len(self.avg_win1_list) > 0 and len(self.avg_win2_list) > 0 and len(self.orders) == 0:
                        if self.avg_win1_list[-1] >= self.avg_win2_list[-1]:
                            # Check that we have enough cash to place the order
                            if self.holdings['CASH'] >= (self.size * ask):
                                self.placeLimitOrder(self.symbol, quantity=self.size, is_buy_order=True, limit_price=ask)
                        else:
                            if self.symbol in self.holdings and self.holdings[self.symbol] > 0:
                                order_size = min(self.size, self.holdings[self.symbol])
                                self.placeLimitOrder(self.symbol, quantity=order_size, is_buy_order=False, limit_price=bid)
            self.setWakeup(currentTime + self.getWakeFrequency())
            self.state = 'AWAITING_WAKEUP'

    def getWakeFrequency(self):
        return pd.Timedelta(self.wake_up_freq)


In [11]:
from agent.TradingAgent import TradingAgent
import pandas as pd
import numpy as np
import os
import pandas as pd
from contributed_traders.util import get_file

class mliu420_blazeit(TradingAgent):
    """
    Mingchun Liu's Market Making Algo
    """

    def __init__(self, id, name, type, symbol, starting_cash, min_size, max_size , wake_up_freq='10s',
                 log_orders=False, random_state=None):

        super().__init__(id, name, type, starting_cash=starting_cash, log_orders=log_orders, random_state=random_state)
        self.symbol = symbol      # Symbol traded
        self.min_size = min_size  # Minimum order size
        self.max_size = max_size  # Maximum order size
        self.size = round(self.random_state.randint(self.min_size, self.max_size) / 2) # order size per LOB side
        self.wake_up_freq = wake_up_freq # Frequency of agent wake up
        self.log_orders = log_orders
        self.state = "AWAITING_WAKEUP"
        # Percentage of the order size to be placed at different levels is determined by levels_quote_dict
        ######################
        self.orders_executed = 0
        self.can_cancel_request = False
        self.paOrders = 0
        #parameters
        self.pricingVolume = 100
        self.depthLevels = 10

    def kernelStarting(self, startTime):
        super().kernelStarting(startTime)

    def wakeup(self, currentTime):
        """ Agent wakeup is determined by self.wake_up_freq """
        can_trade = super().wakeup(currentTime)
        if not can_trade: return
        #check if current time greater than wait time
        if self.cancelCheck(currentTime):
            self.cancelOrders()
        self.getCurrentSpread(self.symbol, depth=self.depthLevels)
        self.state = 'AWAITING_SPREAD'
        self.orders_executed = 0
    
    def receiveMessage(self, currentTime, msg):
        """ Market Maker actions are determined after obtaining the bids and asks in the LOB """
        super().receiveMessage(currentTime, msg)
        try:
            dt = (self.mkt_close - currentTime).totalSeconds()
            if dt < 25:
                self.dump_shares()
                return 0
        except:
            pass

        if msg.body['msg'] == 'ORDER_EXECUTED':
            self.orders_executed += 1
        if msg.body['msg'] == 'ORDER_ACCEPTED':
            self.can_cancel_request = True
        if self.state == 'AWAITING_SPREAD' and msg.body['msg'] == 'QUERY_SPREAD':
            self.calculateAndOrder(currentTime)
            self.setWakeup(currentTime + self.getWakeFrequency())
        #do nothing till other leg executed
        elif self.state == 'AWAITING CONFIRMATION' and msg.body['msg'] == 'ORDER_ACCEPTED':
            self.paOrders -= 1
            if self.paOrders == 0:
                self.state = 'AWAITING EXECUTION'
                self.exec_time_order = currentTime
        elif self.state == 'AWAITING_EXECUTION' and msg.body['msg'] == 'ORDER_EXECUTED':
            #use a condition to see if holdings close to reduce exposure to JPM
            #self.fOrderTime = currentTime
            if len(self.orders) == 0:
                self.setWakeup(currentTime + self.getWakeFrequency())
                self.orders_executed = 0
            elif self.cancelCheck:
                self.cancelOrders()
        elif msg.body['msg'] == 'ORDER_CANCELLED':
            if len(self.orders) == 0:
                self.orders_executed = 0
                self.can_cancel_request = False
                self.setWakeup(currentTime + self.getWakeFrequency())
    
    def cancelOrders(self):
        """ cancels all resting limit orders placed by the market maker """
        for _, order in self.orders.items():
            self.cancelOrder(order)
        self.can_cancel_request = False
            
    def cancelCheck(self, currentTime):
        if self.orders and self.can_cancel_request:
            if self.orders_executed == 0:
                return True
            else:
                try:
                    if int(currentTime - self.exec_time_order).totalSeconds() >= 5:
                        return True
                except:
                    self.exec_time_order = currentTime
        return False
    
    def calculateAndOrder(self, currentTime):
        bid, ask = self.getKnownBidAsk(self.symbol, best=False)
        if bid and ask:
            sumBid = 0
            sumBidVol = 0
            sumAsk = 0
            sumAskVol = 0
            try:
                for i in range(self.depthLevels):
                    if sumBidVol < 100:
                        if sumBidVol + bid[i][1] > self.pricingVolume:
                            sumBidVol = self.pricingVolume
                            sumBid += (self.pricingVolume - bid[i][1]) * bid[i][0]
                        else:
                            sumBid += bid[i][1] * bid[i][0]
                    if sumAskVol < 100:
                        if sumAskVol + ask[i][1] > self.pricingVolume:
                            sumAskVol = self.pricingVolume
                            sumAsk += (self.pricingVolume - ask[i][1]) * ask[i][0]
                        else:
                            sumAsk += ask[i][1] * ask[i][0]

                
                if sumBid == sumAsk:
                    if sumBid == self.pricingVolume:
                        askP = sumAsk / self.pricingVolume
                        bidP = sumBid / self.pricingVolume
                        self.placeLimitOrder(self.symbol, )
                        askVol = self.holdings['CASH'] / askP + max(0, self.holdings[self.symbol])
                        bidVol = self.holdings['CASH'] / bidP + max(0, -self.holdings[self.symbol])
                        self.placeLimitOrder(self.symbol, bidVol, True, bidP)
                        self.paOrders += 1
                        self.placeLimitOrder(self.symbol, askVol, False, askP)
                        self.paOrders += 1
                        self.state = 'AWAITING_CONFIRMATION' #place orders and await execution
            except:
                self.setWakeup(currentTime + self.getWakeFrequency())
                
    def dump_shares(self):
        # get rid of any outstanding shares we have
        if self.symbol in self.holdings and len(self.orders) == 0:
            
            bid, _, ask, _ = self.getKnownBidAsk(self.symbol)
            order_size = self.holdings[self.symbol]
            if order_size > 0
                if bid:
                    self.placeLimitOrder(self.symbol, quantity=order_size, is_buy_order=False, limit_price=0)
                elif ask:
                    self.placeLimitOrder(self.symbol, quantity=abs(order_size), is_buy_order=True, limit_price=0)
    def getWakeFrequency(self):
        return pd.Timedelta(self.wake_up_freq)
    def dump_shares(self):
    # get rid of any outstanding shares we have
        if self.symbol in self.holdings and len(self.orders) == 0:
            order_size = self.holdings[self.symbol]
            bid, _, ask, _ = self.getKnownBidAsk(self.symbol)
            if bid:
                self.placeLimitOrder(self.symbol, quantity=order_size, is_buy_order=False, limit_price=0)


In [ ]:
from agent.TradingAgent import TradingAgent
import pandas as pd
import math

class mliu420_blazeit(TradingAgent):
    """
    This agent was built on the market maker agent with some caveats.
    Prices are determined by a variable pricingVolume. Roughly the price
    of a stock is going to be the average price if to buy/sell pricingVolume
    amount of stock. This average between the buy and sell for 100 shares
    is the price.
    
    My first iteration of this agent tried to be fancy with calculations
    of price happening at different times to speed up the order placing process.
    This caused issues with too many iterations.
    """

    def __init__(self, id, name, type, symbol, starting_cash, min_size, max_size , wake_up_freq='10s',
                 log_orders=False, random_state=None):

        super().__init__(id, name, type, starting_cash=starting_cash, log_orders=log_orders, random_state=random_state)
        self.symbol = symbol      # Symbol traded
        self.min_size = min_size  # Minimum order size
        self.max_size = max_size  # Maximum order size
        self.size = round(self.random_state.randint(self.min_size, self.max_size) / 2) # order size per LOB side
        self.wake_up_freq = wake_up_freq # Frequency of agent wake up
        self.log_orders = log_orders
        self.state = "AWAITING_WAKEUP"
        #parameters
        self.pricingVolume = 100
        self.depthLevels = 10
        self.starting_cash = starting_cash
        self.pOrders = 0

    def kernelStarting(self, startTime):
        super().kernelStarting(startTime)

    def wakeup(self, currentTime):
        """ Agent wakeup is determined by self.wake_up_freq """
        can_trade = super().wakeup(currentTime)
        if not can_trade: return
        self.cancelOrders()
        self.getCurrentSpread(self.symbol, depth=self.depthLevels)
        self.state = 'AWAITING_SPREAD'
        print('true holdings??')
        print(self.holdings)

    def receiveMessage(self, currentTime, msg):
        """ Market Maker actions are determined after obtaining the bids and asks in the LOB """
        super().receiveMessage(currentTime, msg)
        if self.state == 'AWAITING_SPREAD' and msg.body['msg'] == 'QUERY_SPREAD':
            self.calculateAndOrder(currentTime)
#         if msg.body['msg'] = 'ORDER_ACCEPTED':
#             self.pOrders -= 1
        print(msg)
    def cancelOrders(self):
        """ cancels all resting limit orders placed by the market maker """
        for _, order in self.orders.items():
            self.cancelOrder(order)
            
    def calculateAndOrder(self, currentTime):
        bid, ask = self.getKnownBidAsk(self.symbol, best=False)
        print('got bid and ask')
        if bid and ask:
            print('bid and ask')
            print(bid)
            print(ask)
            sumBid = 0
            sumBidVol = 0
            sumAsk = 0
            sumAskVol = 0
            try:
                for i in range(self.depthLevels):
                    if sumBidVol < 100:
                        if sumBidVol + bid[i][1] > self.pricingVolume:
                            sumBid += (self.pricingVolume - sumBidVol) * bid[i][0]
                            sumBidVol = self.pricingVolume
                        else:
                            sumBid += bid[i][1] * bid[i][0]
                            sumBidVol += bid[i][1]
                    if sumAskVol < 100:
                        if sumAskVol + ask[i][1] > self.pricingVolume:
                            sumAsk += (self.pricingVolume - sumAskVol) * ask[i][0]
                            sumAskVol = self.pricingVolume
                        else:
                            sumAsk += ask[i][1] * ask[i][0]
                            sumAskVol += ask[i][1]
                            
                            
                if sumBidVol == sumAskVol:
                    if sumBidVol == self.pricingVolume:
                        askP = sumAsk / self.pricingVolume
                        bidP = sumBid / self.pricingVolume
                        bidVol = math.floor(max(0, self.starting_cash) / bidP / 2)
                        askVol = math.floor(max(0, self.starting_cash) / askP / 2)
                        try:
                            print('bidvol, askvol, jpm, cash',bidVol, askVol, self.holdings[self.symbol],self.holdings['CASH'])
                            bidVol = max(0,bidVol - self.holdings[self.symbol])
                            askVol = max(0,askVol + self.holdings[self.symbol])
                            print('bidvol, askvol, jpm',bidVol, askVol, self.holdings)
                        except:
                            pass
                        askP = round((askP * bidVol + (0.5 * bidP + 0.5 * askP) * askVol) / (bidVol + askVol))
                        bidP = round(2*(bidP * askVol + (0.5 * bidP + 0.5 * askP) * bidVol) / (bidVol + askVol))
                        if bidVol > 0:
                            self.placeLimitOrder(self.symbol, bidVol, True, bidP)
                            #self.pOrders += 1
                        if askVol > 0:
                            self.placeLimitOrder(self.symbol, askVol, False, askP)
                            #self.pOrders += 1
                        print('placed order')
            except Exception as e:
                print(e)
                pass
            self.state = 'AWAITING_WAKEUP' #place orders and await execution
            self.setWakeup(currentTime + self.getWakeFrequency())
    def getWakeFrequency(self):
        return pd.Timedelta(self.wake_up_freq)

In [2]:
x = {'hello':'yese'}
if x:
    print('yes')

yes


In [3]:
try:
    print(xx)
except:
    print('wow')

wow


In [3]:
z = {'hello':-100}
print(max(0,-z['hello']))
print(z['test'])

100


KeyError: 'test'

In [5]:
print(math.floor(1.1))

NameError: name 'math' is not defined